In [1]:
import keras
from keras.preprocessing import sequence
from keras.models import Sequential, load_model, Model
from keras.layers import LSTM, Dense, Dropout, Activation, Embedding, TimeDistributed, Concatenate, Input
from keras.callbacks import EarlyStopping
import word2vec_utils as w2v
import data
import numpy as np
import re
from data_utils import split_dataset 
from chat_constants import *


Using Theano backend.


In [2]:
##### TO RUN Set blank to be 300*[0] + [1]
##### Set w2v to google embedings before running the whole script
assert( EMBED_DIM == 301 )
w2v_model = w2v.initialize()
assert( len( w2v_model.vocab) == 3000000 )

In [3]:
def null_punisher(y_true, y_pred ):
    '''Provides stronger incentive to avoid Null'''
    L = 3
    error = keras.losses.cosine_proximity( y_true,y_pred)
    if y_true != BLANK:
        error *= L
    return error

In [4]:
LSTM_DROPOUT = 0.1
AB_Input = Input(shape =(MAX_SENT_LENGTH*2,EMBED_DIM))

AB_layer1 = LSTM(EMBED_DIM,return_sequences=True, name = "AB_layer1", dropout=LSTM_DROPOUT, recurrent_dropout=LSTM_DROPOUT)
AB_layer2 = LSTM(EMBED_DIM,return_sequences=True, name = "AB_layer2", dropout=LSTM_DROPOUT, recurrent_dropout=LSTM_DROPOUT)
AB_layer3 = LSTM(EMBED_DIM,return_sequences=True, name = "AB_layer3", dropout=LSTM_DROPOUT, recurrent_dropout=LSTM_DROPOUT)

AB_output = AB_layer3(AB_layer2(AB_layer1(AB_Input)))

chat_model = Model(inputs = [ AB_Input], outputs = [AB_output])
chat_model.compile(loss=null_punisher,
              optimizer='Adam',
              metrics=['accuracy']
              )

chat_model.load_weights('AB_len60_301dim_w2v_in_and_out.h5',by_name=True)
chat_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 60, 301)           0         
_________________________________________________________________
AB_layer1 (LSTM)             (None, 60, 301)           726012    
_________________________________________________________________
AB_layer2 (LSTM)             (None, 60, 301)           726012    
_________________________________________________________________
AB_layer3 (LSTM)             (None, 60, 301)           726012    
Total params: 2,178,036
Trainable params: 2,178,036
Non-trainable params: 0
_________________________________________________________________


In [5]:
w2v.unvectorize_initialize()
A1, B, A2 = data.load_data()



In [10]:
i = 1
A1_test,B_test,A2_test = w2v.get_training_data(A1[i:i+1],B[i:i+1],A2[i:i+1])
A1B_test = np.concatenate((A1_test,B_test),axis=1)

In [11]:
print(A1[i])
print(B[i])
print(A2[i])

1 9 1 @ Well , I thought we'd start with pronunciation , if that's okay with you .
2 8 1 @ Not the hacking and gagging and spitting part . Please .
1 9 1 @ Okay . . . then how 'bout we try out some French cuisine . Saturday ? Night ?


In [12]:

predicted = chat_model.predict(A1B_test)
predicted_words = w2v.unvectorize_sentence( predicted[0] )
print( predicted_words )

('w2v word-score', u'1', 0.9980672597885132, '\n', 'uk word-score', 'at-arms', 0.24375356828796638)
('w2v word-score', u'@', 0.9984095692634583, '\n', 'uk word-score', 'aboard--you', 0.20282582263920168)
('w2v word-score', u'Not', 0.9961111545562744, '\n', 'uk word-score', "it's-", 0.1997054688543958)
('w2v word-score', u'the', 0.995297908782959, '\n', 'uk word-score', 'pre-empt', 0.21089029170384549)
('w2v word-score', u'hacking', 0.9780434370040894, '\n', 'uk word-score', 'wo-man', 0.22321916042219775)
('w2v word-score', u'Darmer', 0.3125925064086914, '\n', 'uk word-score', 'gutrush', 0.19659303532697145)
('w2v word-score', u'gagging', 0.8653687238693237, '\n', 'uk word-score', "neil's", 0.18443881196043233)
('w2v word-score', u'Regal_Beagle', 0.3066844642162323, '\n', 'uk word-score', 'gutrush', 0.18993413025796788)
('w2v word-score', u'spitting', 0.9315758943557739, '\n', 'uk word-score', 'try-', 0.19629132881455147)
('w2v word-score', u'part', 0.9129929542541504, '\n', 'uk word-sc